# Simulation - New

In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import uuid
from pandarallel import pandarallel
from tqdm.notebook import tqdm
from copy import deepcopy

pandarallel.initialize()

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
from mc_moea import *

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Read position data

In [3]:
data = []
fnames = glob.glob("position/*.dat")
for fname in fnames:
    X = pickle.load(open(fname, "rb"))
    data.append(X)

### Define pattern's probability

In [4]:
p = [.32, .25, .17, .14, .09, .03]

### Stations

In [5]:
station_types

,types,frequency,capacity,cost
0,900 MHz Type I,900,800,1150000
1,900 MHz Type II,900,1200,1500000
2,1800 MHz Type I,1800,850,880000
3,1800 MHz Type II,1800,1250,1220000
4,2600 MHz Type I,2600,800,950000
5,2600 MHz Type II,2600,1300,1350000


### Simulation parameters

In [6]:
max_station = 20
n_population = 500
crossover_p = 0.2
mutation_p = 0.01
elitism_ratio = 0.9
trim_policy = "best"  # "random" or "best"
num_iters = 100
checkpoint_every = 20

### Initial population

In [7]:
stations_l = [generate_station(n) for n in np.random.randint(1,max_station+1, size=n_population)]

solution_pool = [Solution(stations_l[i]) for i in range(n_population)]

solution_df = pd.DataFrame({"solution_id":[x.id for x in solution_pool],
                            "solution_obj": solution_pool,
                            "from_generation":[-1]*n_population
                           })

In [8]:
int(solution_df.shape[0]*elitism_ratio)

450

### Iteration

In [9]:
iteration_best = []
iteration_pattern = []

In [11]:
for i in tqdm(range(0, num_iters)):
    print(f"Iteration {i+1}/{num_iters}")
    data_pattern = np.random.choice(range(len(data)), p=p)
    position = data[data_pattern]
    print(f"\tFitting with pattern {data_pattern}...")
    print(f"\tComputing fitness value...")
    solution_df = compute(solution_df, position)
    solution_df = solution_df.sort_values(by="fitness_value")
    
    new_solution_df = pd.DataFrame()
    if crossover_p:
        print(f"\tCrossover...")
        crossover_df = crossover(solution_df, crossover_p)
        new_solution_df = pd.concat([new_solution_df, crossover_df])
    if mutation_p:
        print(f"\tMutation...")
        mutation_df = mutation(solution_df, mutation_p)
        new_solution_df = pd.concat([new_solution_df, mutation_df])
        
    if new_solution_df.shape[0] != 0:
        print(f"\tComputing fitness value of new solutions...")
        new_solution_df["solution_id"] = new_solution_df["solution_obj"].parallel_apply(lambda x:x.id)
        new_solution_df = compute(new_solution_df, position)
        new_solution_df["from_generation"] = [i]*new_solution_df.shape[0]
    
    print(f"\tTrimming result...")
    solution_df = pd.concat([solution_df, new_solution_df], ignore_index=True).sample(n=n_population).reset_index(drop=True)\
                    if trim_policy=="random" \
                    else \
                    pd.concat([solution_df, new_solution_df], ignore_index=True).sort_values(by="fitness_value").head(n_population).reset_index(drop=True)
    
    if elitism_ratio:
        print(f"\tPerforming elite selection and reinitialize...")
        n_elite = int(solution_df.shape[0] * elitism_ratio)
        n_reinit = solution_df.shape[0] - n_elite
        solution_df = solution_df.sort_values(by="fitness_value").head(n_elite)
        reinit_stations_l = [generate_station(n) for n in np.random.randint(1, max_station+1, size=n_reinit)]
        reinit_pool = [Solution(s) for s in reinit_stations_l]
        reinit_solution_df = pd.DataFrame({"solution_id":[x.id for x in reinit_pool],
                                          "solution_obj": reinit_pool
                                         })
        reinit_solution_df = compute(reinit_solution_df, position)
        reinit_solution_df["from_generation"] = [i]*reinit_solution_df.shape[0]
        solution_df = pd.concat([solution_df, reinit_solution_df], ignore_index=True)
    
    best = deepcopy(solution_df[solution_df['fitness_value']==min(solution_df['fitness_value'])])
    iteration_best.append(best)
    iteration_pattern.append(data_pattern)
    print(f"\tBest fitness of this iteration: {best['fitness_value'].iloc[0]}")
    if i%checkpoint_every == 0:
        fname = f"solutions/solution-{i:04d}.pkl.bz2"
        print(f"\tSaving checkpint at {fname}")
        solution_df.to_pickle(fname, compression="bz2")
    print(f"\tDone!")

  0%|          | 0/100 [00:00<?, ?it/s]

Iteration 1/100
	Computing fitness value...
	Crossover...
	Mutation...
	Computing fitness value of new solutions...
	Trimming result...
	Performing elite selection and reinitialize...
	Best fitness of this iteration: 1.2067696548491982
	Saving checkpint at solutions/solution-0000.pkl.bz2
	Done!
Iteration 2/100
	Computing fitness value...
	Crossover...
	Mutation...
	Computing fitness value of new solutions...
	Trimming result...
	Performing elite selection and reinitialize...
	Best fitness of this iteration: 1.2216360904240657
	Done!
Iteration 3/100
	Computing fitness value...
	Crossover...
	Mutation...
	Computing fitness value of new solutions...
	Trimming result...
	Performing elite selection and reinitialize...
	Best fitness of this iteration: 1.2182315291330077
	Done!
Iteration 4/100
	Computing fitness value...
	Crossover...
	Mutation...
	Computing fitness value of new solutions...
	Trimming result...
	Performing elite selection and reinitialize...
	Best fitness of this iteration: 1

Process ForkPoolWorker-851:
Process ForkPoolWorker-850:
Traceback (most recent call last):
Process ForkPoolWorker-848:
Process ForkPoolWorker-849:
  File "/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process ForkPoolWorker-844:
Process ForkPoolWorker-845:
Process ForkPoolWorker-846:
  File "/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process ForkPoolWorker-847:
  File "/opt/anaconda3/lib/python3.9/multiprocessing/queues.py", line 365, in get
    with self._rlock:
  File "/opt/anaconda3/

KeyboardInterrupt: 

In [13]:
solution_df.groupby("from_generation").count()

,solution_id,solution_obj,signal_strength,construction_cost,num_orphans,fitness_value
from_generation,,,,,,
-1,166,166,166,166,166,166
0,35,35,35,35,35,35
1,89,89,89,89,89,89
2,105,105,105,105,105,105
3,105,105,105,105,105,105
